# Sun2FLOPs Colab Driver

This notebook installs the package, fetches weather data, runs a single simulation, and performs a parameter sweep.

In [ ]:
!pip install -e .

In [ ]:
from sun2flops.config.models import (
    SiteConfig, WeatherConfig, PVConfig, BatteryConfig, GPUConfig,
    GovernorConfig, SimConfig, SweepConfig, FullConfig,
)
from sun2flops.data.nsrdb import load_weather_range
from sun2flops.sim.engine import run_simulation
from sun2flops.sweep.runner import run_sweep
from sun2flops.viz.plots import plot_timeseries
from sun2flops.viz.heatmaps import plot_heatmap
import json
import pandas as pd

In [ ]:
cfg = FullConfig(
    site=SiteConfig(
        name="Texas",
        latitude=31.9686,
        longitude=-99.9018,
        timezone="America/Chicago",
    ),
    weather=WeatherConfig(source="nsrdb", years=[2021]),
    pv=PVConfig(surface_tilt_deg=20, surface_azimuth_deg=180, dc_nameplate_kw=3),
    battery=BatteryConfig(energy_capacity_kwh=10, soc_init=0.5),
    gpu=GPUConfig(),
    governor=GovernorConfig(),
    sim=SimConfig(),
    sweep=SweepConfig(),
)
cfg

In [ ]:
weather = load_weather_range(cfg.site, cfg.weather)
weather.head()

In [ ]:
results, metrics = run_simulation(weather, cfg, single_year=2021)
metrics

In [ ]:
plot_timeseries(results, year=2021)

In [ ]:
sweep_df = run_sweep(weather, cfg)
sweep_df.head()

In [ ]:
plot_heatmap(sweep_df, x="pv_kw", y="batt_kwh", z="utilization")

In [ ]:
results.to_csv("simulation_timeseries.csv")
with open("simulation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)
sweep_df.to_csv("sweep_results.csv", index=False)